In [26]:
import pandas as pd
import random 
import numpy as np
from scipy.spatial import distance_matrix
import time
import warnings
from tqdm import tqdm 
import itertools
warnings.filterwarnings('ignore')

In [27]:
tsp_c = pd.read_csv("TSPC.csv", sep=';', header=None)
tsp_d = pd.read_csv("TSPD.csv", sep=';', header=None)

In [28]:
def get_random_solution():
    return random.sample([i for i in range(0, 200)], 100)

def calulate_total_cost(path):
    total = 0
    nr = len(path)
    for idx, node in enumerate(path):
        total += dm[node][path[(idx+1)%nr]]
    return total

def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)


    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm+temp+temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf
    
    if show: 
        df_dist = pd.DataFrame(dm)
        display(df_dist)
    return dm

In [29]:
def gen_node_exchange_moves(path):
    not_selected = list(set([i for i in range(200)]) - set(path))
    node_moves = []
    for idx, node in enumerate(path):
        for new_node in not_selected:
            new_cost = dm[path[idx-1]][new_node] + dm[path[(idx+1)%100]][new_node] # two new edges
            new_cost -= (dm[path[idx-1]][node] + dm[path[(idx+1)%100]][node]) #
            node_moves.append((new_cost, 'n', idx, new_node)) #cost, type, idx of old move, new_node
    return node_moves

In [30]:
def gen_edge_exchange_moves(path):
    edge_moves = []
    for idx1, idx2 in all_posibble_comb:
        if (idx2+1)%100 != idx1:
            new_cost = dm[path[idx1]][path[(idx2+1)%100]] + dm[path[idx1-1]][path[idx2]] #new edges
            new_cost -= (dm[path[idx1-1]][path[idx1]] + dm[path[idx2]][path[(idx2+1)%100]]) #old edges
            edge_moves.append((new_cost, 'e', idx1, idx2))
    return edge_moves

In [31]:
dm = generate_dm(tsp_c, False)
all_posibble_comb = list(itertools.combinations([i for i in range(100)], 2))

path = get_random_solution()
print(calulate_total_cost(path))
s = time.time()
run = True
while run:
    run = False
    moves = gen_edge_exchange_moves(path) + gen_node_exchange_moves(path)
    moves.sort(key=lambda x:x[0])
    best_move = moves[0]
    if best_move[0] < 0:
        if best_move[1] == 'n':
            idx = best_move[2]
            new_node = best_move[3]
            path[idx] = new_node
            run = True
        else:
            idx1 = best_move[2]
            idx2 = best_move[3]
            path = path[:idx1] + path[idx1:idx2+1][::-1] + path[idx2+1:]
            run = True
        
            
    
    
print(calulate_total_cost(path))
print(time.time() - s)

225877.0
51350.0
5.3437347412109375
